Table 6: Alcohol-specific age-standardised death rates per 100,000 population, deaths registered in Scotland, 2001 to 2016

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/alcoholspecificdeathsintheukmaindataset')
    sheet = scraper.distribution().as_databaker()

In [2]:
tab = next(tab for tab in sheet if tab.name=='Table 6')

In [3]:
observations = tab.excel_ref('D7').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
observations

{<Q12 43.8>, <G14 27.3>, <K15 13.3>, <O8 41.5>, <P20 26.0>, <G22 22.3>, <J16 13.2>, <L19 12.5>, <J10 15.2>, <F9 26.8>, <P18 24.6>, <I22 342.0>, <O9 42.6>, <O12 41.2>, <G16 24.1>, <Q16 35.7>, <P15 29.8>, <L23 12.9>, <I23 331.0>, <J14 15.2>, <L21 13.3>, <K7 13.0>, <O18 26.7>, <O11 39.7>, <K14 13.7>, <Q8 44.2>, <P19 25.5>, <K12 15.3>, <N22 797.0>, <N9 957.0>, <E14 25.9>, <I16 355.0>, <P23 28.2>, <G17 23.2>, <E16 22.8>, <J7 14.5>, <Q19 29.7>, <P17 28.5>, <F16 21.5>, <K9 14.3>, <L11 18.3>, <E13 25.5>, <Q21 29.8>, <E7 26.1>, <O23 30.3>, <G21 20.6>, <N23 789.0>, <D18 968.0>, <O13 37.6>, <G13 26.9>, <D11 1354.0>, <D20 1036.0>, <J15 14.8>, <G18 19.6>, <Q20 30.1>, <N13 904.0>, <D23 1120.0>, <D9 1354.0>, <N19 694.0>, <O19 27.6>, <G8 29.6>, <Q23 32.5>, <E11 27.6>, <O21 27.7>, <P11 37.1>, <L15 16.3>, <P16 31.1>, <Q14 40.3>, <D16 1183.0>, <I9 397.0>, <Q7 41.7>, <I19 308.0>, <E21 19.5>, <E10 27.3>, <D22 1139.0>, <K23 10.4>, <Q15 34.3>, <E20 19.5>, <F13 24.1>, <K20 10.4>, <J19 11.2>, <G12 29.9>, <K21 

In [5]:
Sex = tab.excel_ref('D5').expand(RIGHT).is_not_whitespace()
Sex

{<N5 'Males'>, <I5 'Females'>, <D5 'All Persons'>}

In [6]:
deaths = tab.excel_ref('D6').expand(RIGHT).is_not_blank()
deaths

{<D6 'Deaths'>, <P6 'Lower 95% confidence limit'>, <N6 'Deaths'>, <G6 'Upper 95% confidence limit'>, <L6 'Upper 95% confidence limit'>, <O6 'Rate per 100,000 males1'>, <K6 'Lower 95% confidence limit'>, <E6 'Rate per 100,000 people1'>, <F6 'Lower 95% confidence limit'>, <J6 'Rate per 100,000 females1'>, <Q6 'Upper 95% confidence limit'>, <I6 'Deaths'>}

In [7]:
# lc = tab.excel_ref('D6').expand(RIGHT).filter(contains_string('Lower 95% confidence limit'))
# lc

In [8]:
# uc = tab.excel_ref('F6').expand(RIGHT).filter(contains_string('Upper 95% confidence limit'))
# uc

In [9]:
Year = tab.excel_ref('B7').expand(DOWN) - tab.excel_ref('A24').expand(DOWN)  
Year

{<B24 'Source: National Records of Scotland'>, <B21 2015.0>, <B14 2008.0>, <B11 2005.0>, <B8 2002.0>, <B18 2012.0>, <B15 2009.0>, <B17 2011.0>, <B26 'Notes:'>, <B7 2001.0>, <B19 2013.0>, <B22 2016.0>, <B16 2010.0>, <B9 2003.0>, <B12 2006.0>, <B25 ''>, <B31 ''>, <B27 '1 Age-standardised rates are expressed per 100,000 population, standardised to the 2013 European Standard Population.'>, <B32 ''>, <B33 ''>, <B23 2017.0>, <B13 2007.0>, <B20 2014.0>, <B30 '4 The lower and upper confidence limits have been provided. These form a confidence interval, which is a measure of the statistical precision of an estimate and shows the range of uncertainty around the estimated figure. Calculations based on small numbers of events are often subject to random fluctuations. As a general rule, if the confidence interval around one figure overlaps with the interval around another, we cannot say with certainty that there is more than a chance difference between the two figures.'>, <B10 2004.0>, <B29 '3 Figu

In [10]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDim(deaths, 'Alcohol Specific Deaths',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [11]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [12]:
new_table = c1.topandas()
new_table

,OBS,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,1228.0,2001.0,All Persons,Deaths,Count,People
1,26.1,2001.0,All Persons,"Rate per 100,000 people1",Count,People
2,24.6,2001.0,All Persons,Lower 95% confidence limit,Count,People
3,27.6,2001.0,All Persons,Upper 95% confidence limit,Count,People
4,359.0,2001.0,Females,Deaths,Count,People
5,14.5,2001.0,Females,"Rate per 100,000 females1",Count,People
6,13.0,2001.0,Females,Lower 95% confidence limit,Count,People
7,16.0,2001.0,Females,Upper 95% confidence limit,Count,People
8,869.0,2001.0,Males,Deaths,Count,People
9,39.0,2001.0,Males,"Rate per 100,000 males1",Count,People


In [13]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [14]:
new_table['Year'] = new_table['Year'].astype(int)

In [15]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [16]:
# new_table['Year'] = new_table['Year'].astype(str)

In [17]:
# new_table['Value'] = new_table['Value'].astype(int)

In [18]:
new_table.dtypes

Value                      float64
Year                         int32
Sex                         object
Alcohol Specific Deaths     object
Measure Type                object
Unit                        object
dtype: object

In [19]:
new_table

,Value,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,1228.0,2001,All Persons,Deaths,Count,People
1,26.1,2001,All Persons,"Rate per 100,000 people1",Count,People
2,24.6,2001,All Persons,Lower 95% confidence limit,Count,People
3,27.6,2001,All Persons,Upper 95% confidence limit,Count,People
4,359.0,2001,Females,Deaths,Count,People
5,14.5,2001,Females,"Rate per 100,000 females1",Count,People
6,13.0,2001,Females,Lower 95% confidence limit,Count,People
7,16.0,2001,Females,Upper 95% confidence limit,Count,People
8,869.0,2001,Males,Deaths,Count,People
9,39.0,2001,Males,"Rate per 100,000 males1",Count,People


In [20]:
nt1 = new_table[new_table['Alcohol Specific Deaths'] == 'Lower 95% confidence limit']

In [21]:
nt1

,Value,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
2,24.6,2001,All Persons,Lower 95% confidence limit,Count,People
6,13.0,2001,Females,Lower 95% confidence limit,Count,People
10,36.4,2001,Males,Lower 95% confidence limit,Count,People
14,26.6,2002,All Persons,Lower 95% confidence limit,Count,People
18,14.5,2002,Females,Lower 95% confidence limit,Count,People
22,38.8,2002,Males,Lower 95% confidence limit,Count,People
26,26.8,2003,All Persons,Lower 95% confidence limit,Count,People
30,14.3,2003,Females,Lower 95% confidence limit,Count,People
34,39.9,2003,Males,Lower 95% confidence limit,Count,People
38,25.8,2004,All Persons,Lower 95% confidence limit,Count,People


In [22]:
nt1.columns = ['Lower 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [23]:
nt2 = new_table[new_table['Alcohol Specific Deaths'] == 'Upper 95% confidence limit']

In [24]:
nt2

,Value,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
3,27.6,2001,All Persons,Upper 95% confidence limit,Count,People
7,16.0,2001,Females,Upper 95% confidence limit,Count,People
11,41.7,2001,Males,Upper 95% confidence limit,Count,People
15,29.6,2002,All Persons,Upper 95% confidence limit,Count,People
19,17.7,2002,Females,Upper 95% confidence limit,Count,People
23,44.2,2002,Males,Upper 95% confidence limit,Count,People
27,29.9,2003,All Persons,Upper 95% confidence limit,Count,People
31,17.4,2003,Females,Upper 95% confidence limit,Count,People
35,45.4,2003,Males,Upper 95% confidence limit,Count,People
39,28.8,2004,All Persons,Upper 95% confidence limit,Count,People


In [25]:
nt2.columns = ['Upper 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [26]:
new_table.count()

Value                      204
Year                       204
Sex                        204
Alcohol Specific Deaths    204
Measure Type               204
Unit                       204
dtype: int64

In [27]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Upper 95% confidence limit']

In [28]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Lower 95% confidence limit']

In [29]:
Final_table = pd.merge(new_table, nt1, how = 'inner', on = ['Year','Sex'])

In [30]:
Final_table.head(2)

,Value,Year,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Alcohol Specific Deaths_y,Measure Type_y,Unit_y
0,1228.0,2001,All Persons,Deaths,Count,People,24.6,Lower 95% confidence limit,Count,People
1,26.1,2001,All Persons,"Rate per 100,000 people1",Count,People,24.6,Lower 95% confidence limit,Count,People


In [31]:
Final_table = Final_table[['Value','Year','Sex','Alcohol Specific Deaths_x','Measure Type_x','Unit_x',
                           'Lower 95% confidence limit']]

In [32]:
Final_table = pd.merge(Final_table, nt2, how = 'inner', on = ['Year','Sex'])

In [33]:
Final_table.head(1)

,Value,Year,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Upper 95% confidence limit,Alcohol Specific Deaths,Measure Type,Unit
0,1228.0,2001,All Persons,Deaths,Count,People,24.6,27.6,Upper 95% confidence limit,Count,People


In [34]:
Final_table = Final_table[['Year','Sex','Alcohol Specific Deaths_x','Measure Type','Value','Lower 95% confidence limit', 'Upper 95% confidence limit','Unit']]

In [35]:
Final_table

,Year,Sex,Alcohol Specific Deaths_x,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit
0,2001,All Persons,Deaths,Count,1228.0,24.6,27.6,People
1,2001,All Persons,"Rate per 100,000 people1",Count,26.1,24.6,27.6,People
2,2001,Females,Deaths,Count,359.0,13.0,16.0,People
3,2001,Females,"Rate per 100,000 females1",Count,14.5,13.0,16.0,People
4,2001,Males,Deaths,Count,869.0,36.4,41.7,People
5,2001,Males,"Rate per 100,000 males1",Count,39.0,36.4,41.7,People
6,2002,All Persons,Deaths,Count,1334.0,26.6,29.6,People
7,2002,All Persons,"Rate per 100,000 people1",Count,28.1,26.6,29.6,People
8,2002,Females,Deaths,Count,400.0,14.5,17.7,People
9,2002,Females,"Rate per 100,000 females1",Count,16.1,14.5,17.7,People


In [36]:
Final_table.columns = ['Alcohol Specific Deaths' if x=='Alcohol Specific Deaths_x' else x for x in Final_table.columns]

In [37]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Lower 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Lower 95% confidence limit']), axis = 1)

In [38]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Upper 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Upper 95% confidence limit']), axis = 1)

In [39]:
Final_table['Alcohol Specific Deaths'] = Final_table['Alcohol Specific Deaths'].str.rstrip('1')

In [40]:
Final_table['Sex'] = Final_table['Sex'].map(
    lambda x: {
        'All Persons' : 'T', 
        'Males' : 'M',
        'Females': 'F'         
        }.get(x, x))

In [41]:
Final_table['Age'] =  'All'

In [42]:
Final_table['Geography'] =  'S92000003'

In [43]:
Final_table

,Year,Sex,Alcohol Specific Deaths,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit,Age,Geography
0,2001,T,Deaths,Count,1228.0,,,People,All,S92000003
1,2001,T,"Rate per 100,000 people",Count,26.1,24.6,27.6,People,All,S92000003
2,2001,F,Deaths,Count,359.0,,,People,All,S92000003
3,2001,F,"Rate per 100,000 females",Count,14.5,13,16,People,All,S92000003
4,2001,M,Deaths,Count,869.0,,,People,All,S92000003
5,2001,M,"Rate per 100,000 males",Count,39.0,36.4,41.7,People,All,S92000003
6,2002,T,Deaths,Count,1334.0,,,People,All,S92000003
7,2002,T,"Rate per 100,000 people",Count,28.1,26.6,29.6,People,All,S92000003
8,2002,F,Deaths,Count,400.0,,,People,All,S92000003
9,2002,F,"Rate per 100,000 females",Count,16.1,14.5,17.7,People,All,S92000003


In [44]:
Final_table.count()

Year                          102
Sex                           102
Alcohol Specific Deaths       102
Measure Type                  102
Value                         102
Lower 95% confidence limit    102
Upper 95% confidence limit    102
Unit                          102
Age                           102
Geography                     102
dtype: int64